# SETUP

In [ ]:
import sys
import json
from collections import Counter
import time
import re
import copy
from copy import deepcopy

# GLOBAL VARS

In [ ]:
replay = "100_sampled_replays_edison/58555246.json"

p1_me = list(range(1, 100))
p2_op = list(range(100, 200))

# HELPER FUNCTIONS

In [ ]:
def assign_id(ids, card_name, card_state=None, obj_id=None, battle_position=None, face_up=None):
    first_id = ids.pop(0)
    return {"name": card_name, "id": first_id, "state": card_state, "object_id": obj_id, "battle_position": battle_position, "face_up": face_up}

def my_card(c, obj_id, all_cards):
    found = False
    for card in all_cards:
        if card["name"] == c and obj_id == card.get("object_id"):
            found = True
            break
    return found

def soft_my_card(c, all_cards):
    found = False
    for card in all_cards:
        if card["name"] == c:
            found = True
            break
    return found

def get_log(play):
    return play["log"].get("private_log") or play["log"].get("public_log")

# strip "(x/y)" trailing hints like "(4/5)" or "(1/1)"
_PAREN_HINT_RE = re.compile(r"\s*\(\s*\d+\s*/\s*\d+\s*\)\s*$")

def _strip_hint(s: str) -> str:
    return _PAREN_HINT_RE.sub("", s.strip())

def debug_state_play(state_bef, state_aft, play):
    print("\n=== DEBUG STATE PLAY ===")
    print("------------------ BEFORE:")
    print(json.dumps(state_bef, indent=2))
    print("------------------ AFTER:")
    print(json.dumps(state_aft, indent=2))
    print("------------------ PLAY:")
    print(json.dumps(play, indent=2))
    print("=======================\n")

def decompose_public_log(log: str):
    s = log.strip()

    # 1) Full: ... "<card>" from <start> to <finish>
    m = re.match(
        r'^(?P<action>.+?)\s+"(?P<card>[^"]+)"\s+from\s+(?P<start>.+?)\s+to\s+(?P<finish>.+)$',
        s, flags=re.IGNORECASE
    )
    if m:
        action = m.group("action").strip()
        card_name = m.group("card").strip()
        start_location = _strip_hint(m.group("start"))
        finish_location = _strip_hint(m.group("finish"))
        return action, card_name, start_location, finish_location

    # 2) Banished shortcut: ... banished "<card>" (x/y)? to <finish>  (no explicit 'from')
    m = re.match(
        r'^(?P<action>.+?)\s+banished\s+"(?P<card>[^"]+)"(?:\s*\(\s*\d+\s*/\s*\d+\s*\))?\s+to\s+(?P<finish>.+)$',
        s, flags=re.IGNORECASE
    )
    if m:
        action = m.group("action").strip()
        card_name = m.group("card").strip()
        start_location = "banished"
        finish_location = _strip_hint(m.group("finish"))
        return action, card_name, start_location, finish_location
    
    # 3) NEW: "in ... to ..." form
    m = re.match(
        r'^(?P<action>.+?)\s+"(?P<card>[^"]+)"\s+in\s+(?P<start>.+?)\s+to\s+(?P<finish>.+)$',
        s, flags=re.IGNORECASE
    )
    if m:
        action = m.group("action").strip()
        card_name = m.group("card").strip()
        start_location = _strip_hint(m.group("start"))
        finish_location = _strip_hint(m.group("finish")).lower()
        return action, card_name, start_location, finish_location
    
    # 3.5) NEW CASE: Summoned a token in ...
    m = re.match(
        r'^(?P<action>summoned\s+a\s+token)\s+in\s+(?P<loc>.+)$',
        s, flags=re.IGNORECASE
    )
    if m:
        action = m.group("action").strip()
        card_name = "token"
        start_location = None
        finish_location = _strip_hint(m.group("loc"))
        return action, card_name, start_location, finish_location
    
    # 4) "in" form: ... "<card>" in <loc>
    m = re.match(
        r'^(?P<action>.+?)\s+"(?P<card>[^"]+)"\s+in\s+(?P<loc>.+)$',
        s, flags=re.IGNORECASE
    )
    if m:
        action = m.group("action").strip()
        card_name = m.group("card").strip()
        start_location = _strip_hint(m.group("loc"))
        finish_location = None
        return action, card_name, start_location, finish_location

    # 5) "to"-only: ... "<card>" to <finish>
    m = re.match(
        r'^(?P<action>.+?)\s+"(?P<card>[^"]+)"(?:\s*\(\s*\d+\s*/\s*\d+\s*\))?\s+to\s+(?P<finish>.+)$',
        s, flags=re.IGNORECASE
    )
    if m:
        action = m.group("action").strip()
        card_name = m.group("card").strip()
        start_location = None
        finish_location = _strip_hint(m.group("finish"))
        return action, card_name, start_location, finish_location

    # 6) NEW: "from"-only: ... "<card>" from <start>
    m = re.match(
        r'^(?P<action>.+?)\s+"(?P<card>[^"]+)"\s+from\s+(?P<start>.+?)$',
        s, flags=re.IGNORECASE
    )
    if m:
        action = m.group("action").strip()
        card_name = m.group("card").strip()
        start_location = _strip_hint(m.group("start"))
        # Infer destination when action implies it
        finish_location = "banished" if "banish" in action.lower() else None
        return action, card_name, start_location, finish_location

    print("No match found.")
    return None

In [ ]:
def json_diff(before, after, path=""):
    """Recursively compare two JSON-like structures and return their differences."""
    diffs = {}

    # Keys that exist in either dict
    all_keys = set(before.keys()) | set(after.keys())
    for key in all_keys:
        current_path = f"{path}.{key}" if path else key

        if key not in before:
            diffs[current_path] = {"added": after[key]}
        elif key not in after:
            diffs[current_path] = {"removed": before[key]}
        else:
            val_before, val_after = before[key], after[key]

            if isinstance(val_before, dict) and isinstance(val_after, dict):
                nested = json_diff(val_before, val_after, current_path)
                diffs.update(nested)
            elif isinstance(val_before, list) and isinstance(val_after, list):
                if val_before != val_after:
                    diffs[current_path] = {
                        "before": val_before,
                        "after": val_after,
                    }
            elif val_before != val_after:
                diffs[current_path] = {
                    "before": val_before,
                    "after": val_after,
                }

    return diffs


# GENERAL ANALYSIS

In [ ]:
def parse(content):
    persistent_state = {
        # general
        "turn": 0,
        ### me
        "me_life": 8000,
        "me_hand": [],
        "me_GY": [],
        "me_banished": [],
        "me_field": {
            "M-1": None,
            "M-2": None,
            "M-3": None,
            "M-4": None,
            "M-5": None,
            "S-1": None,
            "S-2": None,
            "S-3": None,
            "S-4": None,
            "S-5": None,
            "FSZ": None,
        },
        "me_phase": None,
        ### opponent
        "op_life": 8000,
        "op_hand": [],
        "op_GY": [],
        "op_banished": [],
        "op_field": {
            "M-1": None,
            "M-2": None,
            "M-3": None,
            "M-4": None,
            "M-5": None,
            "S-1": None,
            "S-2": None,
            "S-3": None,
            "S-4": None,
            "S-5": None,
            "FSZ": None,
        },
        "op_phase": None,
    }
    p1 = content["player1"]["username"]
    p2 = content["player2"]["username"]

    for i, play in enumerate(content["plays"]):
        #skip any play that doesnt change the state
        if (((play["play"] == "Duel message" or play["play"] == "Shuffle hand" 
                or play["play"] == "Good" or play["play"] == "Stop viewing") 
                or play["play"] == "Shuffle deck") or play["play"] == "Declare"
                or play["play"] == "View ED" or play["play"] == "View GY"
                or play["play"] == "View deck" or play["play"] == "Attack directly"
                or play["play"] == "Edit stats" or play["play"] == "Attack"
                or play["play"] == "View GY 2" or play["play"] == "View Banished"
                or play["play"] == "Thinking" or play["play"] =="Target card"
                or play["play"] == "Add counter" or play["play"] == 'RPS'
                or play["play"] == "Add watcher" or play["play"] == "Remove watcher"):
            continue
        before_state = deepcopy(persistent_state)

        # print(f"Doing play {i}: {play}")
        if play["play"] == "Admit defeat" or play["play"] == "Left duel":
            print("Admit Defeat found, exiting parse.")
            break
        
        if play["play"] == "Life points":
            if play["username"] == p1:
                amount = play["amount"]
                persistent_state["me_life"] += amount
            else:
                amount = play["amount"]
                persistent_state["op_life"] += amount

        if play["play"] == "Pick first":
            cards = [(card["name"], card["object_id"]) for card in play["cards"]]
            cards_p1 = cards[:5]
            cards_p2 = cards[5:10]
            persistent_state["me_hand"] = [assign_id(p1_me, card, "private", obj_id, battle_position=None, face_up=None) for card, obj_id in cards_p1]
            persistent_state["op_hand"] = [assign_id(p2_op, card, "private", obj_id, battle_position=None, face_up=None) for card, obj_id in cards_p2]

        if play["play"] == "Draw card":
            if play["username"] == p1:
                card_name = play["card"]["name"]
                obj_id = play["card"]["object_id"]
                persistent_state["me_hand"].append(assign_id(p1_me, card_name, "private", obj_id, battle_position=None, face_up=None))
            else:
                card_name = play["card"]["name"]
                obj_id = play["card"]["object_id"]
                persistent_state["op_hand"].append(assign_id(p2_op, card_name, "private", obj_id, battle_position=None, face_up=None))
            
        if play["play"] == "To hand":
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            if play["username"] == p1:
                card_name = play["card"]["name"]
                obj_id = play["card"]["object_id"]
                persistent_state["me_hand"].append(assign_id(p1_me, card_name, "public", obj_id, battle_position=None, face_up=None))
                side = "me"
            else:
                card_name = play["card"]["name"]
                obj_id = play["card"]["object_id"]
                persistent_state["op_hand"].append(assign_id(p2_op, card_name, "public", obj_id, battle_position=None, face_up=None))
                side = 'op'

            field_key = f"{side}_field"
            # removing from field(like Swap frog eff)
            if start_location in persistent_state[field_key]:
                persistent_state[field_key][start_location] = None

        if play["play"] == "Enter DP":
            if play["username"] == p1:
                persistent_state["me_phase"] = "DP"
                persistent_state["op_phase"] = None
            else:
                persistent_state["op_phase"] = "DP"
                persistent_state["me_phase"] = None

        if play["play"] == "Enter SP":
            if play["username"] == p1:
                persistent_state["me_phase"] = "SP"
                persistent_state["op_phase"] = None
            else:
                persistent_state["op_phase"] = "SP"
                persistent_state["me_phase"] = None

        if play["play"] == "Enter M1":
            if play["username"] == p1:
                persistent_state["me_phase"] = "M1"
                persistent_state["op_phase"] = None
            else:
                persistent_state["op_phase"] = "M1"
                persistent_state["me_phase"] = None

        if play["play"] == "Enter BP":
            if play["username"] == p1:
                persistent_state["me_phase"] = "BP"
                persistent_state["op_phase"] = None
            else:
                persistent_state["op_phase"] = "BP"
                persistent_state["me_phase"] = None

        if play["play"] == "Enter M2":
            if play["username"] == p1:
                persistent_state["me_phase"] = "M2"
                persistent_state["op_phase"] = None
            else:
                persistent_state["op_phase"] = "M2"
                persistent_state["me_phase"] = None
                
        if play["play"] == "Enter EP":
            if play["username"] == p1:
                persistent_state["me_phase"] = "EP"
                persistent_state["op_phase"] = None
            else:
                persistent_state["op_phase"] = "EP"
                persistent_state["me_phase"] = None

        if play["play"] == "End turn":
            persistent_state["turn"] += 1
            if play["username"] == p1:
                persistent_state["me_phase"] = None
                persistent_state["op_phase"] = "DP"
            else:
                persistent_state["op_phase"] = None
                persistent_state["me_phase"] = "DP"

        if play["play"] == "Activate Field Spell":
            card = play["card"]["name"]
            obj_id = play["card"]["object_id"]
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            print(f"Action={action}, Card={card_name}, From={start_location}, To={finish_location}")
            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = my_card(card, obj_id, all_me_cards)

            if play["username"] == p1 and is_my_card:
                side = "me"
            elif play["username"] == p2 and not is_my_card:
                side = "op"
            else:
                raise ValueError("FIELD SPELL - unexpected player/card ownership")
            
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"
            banished_key = f"{side}_banished"

            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location == "Field Spell Zone":
                source = [persistent_state[field_key]["FSZ"]]
            else:
                print(start_location, "Field Spell Zone"==start_location)
                raise ValueError("FIELD SPELL - unhandled start_location")
            
            card_entry = next(
                (c for c in source if c["name"] == card and c["object_id"] == obj_id),
                None
            )

            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"

            # update and move card
            card_entry["state"] = "public"
            card_entry["face_up"] = True

            # remove from hand if necessary
            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
            elif start_location == "Field Spell Zone":
                pass
            else:
                raise ValueError("FIELD SPELL - something fishy happened")

            persistent_state[field_key]["FSZ"] = card_entry
                  
        if play["play"] == "Set Field Spell":
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            print(f"Action={action}, Card={card_name}, From={start_location}, To={finish_location}")
            if play["username"] == p1:
                side = "me"
            elif play["username"] == p2:
                side = "op"
            else:
                raise ValueError("SET FIELD SPELL - unexpected player/card ownership")
            
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"

            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location == "Field Spell Zone":
                source = [persistent_state[field_key]["FSZ"]]
            else:
                print(start_location, "Field Spell Zone"==start_location)
                raise ValueError("SET FIELD SPELL - unhandled start_location")
            
            print("Source for SET FIELD SPELL:", source)
            card_entry = next(
                (c for c in source if c["name"] == card_name),
                None
            )

            assert card_entry is not None, f"Card {card_name} not found in source in play {i}"
            # update and move card
            card_entry["state"] = "private"
            card_entry["face_up"] = False


            if start_location == "hand":
                obj_id = card_entry.get("object_id", None)
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
            elif start_location == "Field Spell Zone":
                pass
            else:
                raise ValueError("SET FIELD SPELL - something fishy happened")
            
            persistent_state[field_key]["FSZ"] = card_entry

        if play["play"] == "Activate ST":
            card = play["card"]["name"]
            obj_id = play["card"]["object_id"]
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            assert card == card_name, f"Card name mismatch in play {i}: {card} vs {card_name}"
            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = my_card(card, obj_id, all_me_cards)

            if play["username"] == p1 and is_my_card:
                side = "me"
            elif play["username"] == p2 and not is_my_card:
                side = "op"
            else:
                raise ValueError("ACTIVATE ST - unexpected player/card ownership")
            
            # Programatic keys
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"

            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location in persistent_state[field_key]:
                # field is a dict of zones, flatten values for search
                source = [c for c in persistent_state[field_key].values() if c is not None]
            else:
                raise ValueError("ACTIVATE ST - unhandled start_location")
            
    
            card_entry = next(
                (c for c in source if c["name"] == card and c["object_id"] == obj_id),
                None
            )

            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"

            # update and move card
            card_entry["state"] = "public"
            card_entry["face_up"] = True
            if finish_location:
                persistent_state[field_key][finish_location] = card_entry

            # remove from hand if necessary
            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
        
        if play["play"] == "Set ST":
            card_object = play.get("card", None)
            card = card_object["name"] if card_object else None
            obj_id = card_object["object_id"] if card_object else None
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = soft_my_card(card_name, all_me_cards)

            if play["username"] == p1 and is_my_card:
                side = "me"
            elif play["username"] == p2 and not is_my_card:
                side = "op"
            else:
                #needed if card was already known
                is_my_card = my_card(card_name, obj_id, all_me_cards)
                if play["username"] == p1 and is_my_card:
                    side = "me"
                elif play["username"] == p2 and not is_my_card:
                    side = "op"
                else:
                    raise ValueError("Set ST - unexpected player/card ownership")

            
            # Programatic keys
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"


            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location in persistent_state[field_key]:
                # field is a dict of zones, flatten values for search
                source = [c for c in persistent_state[field_key].values() if c is not None]
            else:
                raise ValueError("Set ST- unhandled start_location")
            
            card_entry = next(
                (c for c in source if c["name"] == card_name),
                None
            )

            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"
            
            # update and move card
            card_entry["state"] = "private"
            card_entry["face_up"] = False
            if finish_location:
                persistent_state[field_key][finish_location] = card_entry

            obj_id = card_entry.get("object_id", None)
            

            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]

        if play["play"] == "To ST":
            card_object = play.get("card", None)
            card = card_object["name"] if card_object else None
            obj_id = card_object["object_id"] if card_object else None
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = soft_my_card(card_name, all_me_cards)

            if play["username"] == p1 and is_my_card:
                side = "me"
            elif play["username"] == p2 and not is_my_card:
                side = "op"
            else:
                #needed if card was already known
                is_my_card = my_card(card_name, obj_id, all_me_cards)
                if play["username"] == p1 and is_my_card:
                    side = "me"
                elif play["username"] == p2 and not is_my_card:
                    side = "op"
                else:
                    raise ValueError("Set ST - unexpected player/card ownership")

            hand_key = f"{side}_hand"
            gy_key = f"{side}_GY"
            banished_key = f"{side}_banished"

            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location == "GY":
                source = persistent_state[gy_key]
            elif start_location == "banished":
                source = persistent_state[banished_key]
            elif start_location == "Extra Deck" or start_location == "Deck":
                card_name = play["card"]["name"]
                obj_id = play["card"]["object_id"]
            else:
                raise ValueError("Set ST- unhandled start_location")
            
            if start_location != "Extra Deck" and start_location != "Deck":
                card_entry = next(
                    (c for c in source if c["name"] == card and c["object_id"] == obj_id),
                    None
                )
            else:
                card_entry = assign_id(p1_me, card_name, "public", obj_id, battle_position=None, face_up=None)

            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"
            # update and move card

            card_entry["state"] = "public"
            card_entry["face_up"] = True
            if finish_location:
                persistent_state[field_key][finish_location] = card_entry
            # remove from source if necessary
            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
            elif start_location == "GY":
                persistent_state[gy_key] = [
                    c for c in persistent_state[gy_key] if c["object_id"] != obj_id
                ]
            elif start_location == "banished":
                persistent_state[banished_key] = [
                    c for c in persistent_state[banished_key] if c["object_id"] != obj_id
                ]

        if play["play"] == "Move":
            card_object = play.get("card", None)
            card = card_object["name"] if card_object else None
            obj_id = card_object["object_id"] if card_object else None
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = soft_my_card(card_name, all_me_cards)

            if play["username"] == p1 and is_my_card:
                side = "me"
            elif play["username"] == p2 and not is_my_card:
                side = "op"
            else:
                #needed if card was already known
                is_my_card = my_card(card_name, obj_id, all_me_cards)
                if play["username"] == p1 and is_my_card:
                    side = "me"
                elif play["username"] == p2 and not is_my_card:
                    side = "op"
                else:
                    raise ValueError("Set ST - unexpected player/card ownership")
                
            print(f"Move detected: {card_name} from {start_location} to {finish_location} for player {side}")
            print(play)
            # if finish_location.split("-")[0] == "M" or finish_location.split("-")[0] == "S":
            #     if start_location.split("-")[0] == "M" or start_location.split("-")[0] == "S":
            #         # moving between my zones
            ops_start = len(start_location.split("-")[0]) == 2
            ops_finish = len(finish_location.split("-")[0]) == 2
            print(f"Ops start: {ops_start}, Ops finish: {ops_finish}")
            if ops_start != ops_finish:
                print("Changing sides")
                if ops_start:
                    field_key_from = f"op_field"
                    field_key_to = f"me_field"
                else:
                    field_key_from = f"me_field"
                    field_key_to = f"op_field"
                start_p1 = start_location.split("-")[0]
                finish_p1 = finish_location.split("-")[0]
                # first letter 
                start_p1 = "M" if start_p1[0] == "M" else "S"
                finish_p1 = "M" if finish_p1[0] == "M" else "S"
                start_location = f"{start_p1}-{start_location.split('-')[1]}"
                finish_location = f"{finish_p1}-{finish_location.split('-')[1]}"
                print(f"From key: {field_key_from}, loc: {start_location}")
                print(f"To key: {field_key_to}, loc: {finish_location}")
                persistent_state[field_key_to][finish_location] = persistent_state[field_key_from][start_location]
                persistent_state[field_key_from][start_location] = None
            else:
                print("No changing sides")
                field_key = f"{side}_field"
                persistent_state[field_key][finish_location] = persistent_state[field_key][start_location]
                persistent_state[field_key][start_location] = None

        if play["play"] == "Set monster":
            card_object = play.get("card", None)
            card = card_object["name"] if card_object else None
            obj_id = card_object["object_id"] if card_object else None
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = soft_my_card(card_name, all_me_cards)

            if play["username"] == p1 and is_my_card:
                side = "me"
            elif play["username"] == p2 and not is_my_card:
                side = "op"
            else:
                raise ValueError("ACTIVATE ST - unexpected player/card ownership")
            
            # Programatic keys
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"


            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location in persistent_state[field_key]:
                # field is a dict of zones, flatten values for search
                source = [c for c in persistent_state[field_key].values() if c is not None]
            else:
                raise ValueError("ACTIVATE ST - unhandled start_location")
            
            card_entry = next(
                (c for c in source if c["name"] == card_name),
                None
            )

            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"
            # Setting doesnt have zone in play object
            # zone = play.get("zone", None)
            # assert zone == finish_location, f"Zone mismatch in play {i}: {zone} vs {finish_location}"
            # update and move card
            card_entry["state"] = "private"
            card_entry["battle_position"] = "DEF"
            card_entry["face_up"] = False
            if finish_location:
                persistent_state[field_key][finish_location] = card_entry

            obj_id = card_entry.get("object_id", None)
            

            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
        
        if play["play"] == "Normal Summon":
            card = play["card"]["name"]
            obj_id = play["card"]["object_id"]
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            assert card == card_name, f"Card name mismatch in play {i}: {card} vs {card_name}"

            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = my_card(card, obj_id, all_me_cards)

            if play["username"] == p1 and is_my_card:
                side = "me"
            elif play["username"] == p2 and not is_my_card:
                side = "op"
            else:
                raise ValueError("ACTIVATE ST - unexpected player/card ownership")
            
            # Programatic keys
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"

            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location in persistent_state[field_key]:
                # field is a dict of zones, flatten values for search
                source = [c for c in persistent_state[field_key].values() if c is not None]
            else:
                raise ValueError("ACTIVATE ST - unhandled start_location")
            
    
            card_entry = next(
                (c for c in source if c["name"] == card and c["object_id"] == obj_id),
                None
            )

            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"
            
            card_entry["state"] = "public"
            card_entry["battle_position"] = "ATK"
            card_entry["face_up"] = True

            if finish_location:
                persistent_state[field_key][finish_location] = card_entry

            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
        
        if play["play"] == 'SS ATK':
            card = play["card"]["name"]
            obj_id = play["card"]["object_id"]
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            assert card == card_name, f"Card name mismatch in play {i}: {card} vs {card_name}"

            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = my_card(card, obj_id, all_me_cards)

            if play["username"] == p1:
                side = "me"
            elif play["username"] == p2:
                side = "op"
            else:
                raise ValueError("SS ATK - unexpected player/card ownership")
            
            # Programatic keys
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"
            banished_key = f"{side}_banished"
            gy_key = f"{side}_GY"

            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location in persistent_state[field_key]:
                # field is a dict of zones, flatten values for search
                source = [c for c in persistent_state[field_key].values() if c is not None]
            elif start_location == "Extra Deck" or start_location == "Deck":
                card_name = play["card"]["name"]
                obj_id = play["card"]["object_id"]
            elif start_location == "GY":
                source = persistent_state[gy_key]
            elif start_location == "banished":
                source = persistent_state[banished_key]
            else:
                raise ValueError("SS ATK - unhandled start_location")
            
            if start_location != "Extra Deck" and start_location != "Deck":
                card_entry = next(
                    (c for c in source if c["name"] == card and c["object_id"] == obj_id),
                    None
                )
            else:
                card_entry = assign_id(p1_me, card_name, "public", obj_id, battle_position=None, face_up=None)
                

            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"
            
            card_entry["state"] = "public"
            card_entry["face_up"] = True
            card_entry["battle_position"] = "ATK"


            if finish_location:
                #once decompose is better split will not be needed
                persistent_state[field_key][finish_location.split(' ')[0]] = card_entry


            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
            if start_location == "GY":
                persistent_state[gy_key] = [
                    c for c in persistent_state[gy_key] if c["object_id"] != obj_id
                ]
            if start_location == "banished":
                persistent_state[banished_key] = [
                    c for c in persistent_state[banished_key] if c["object_id"] != obj_id
                ]
        
        if play["play"] == 'SS DEF':
            card = play["card"]["name"]
            obj_id = play["card"]["object_id"]
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            assert card == card_name, f"Card name mismatch in play {i}: {card} vs {card_name}"

            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = soft_my_card(card, all_me_cards)

            if play["username"] == p1:
                side = "me"
            elif play["username"] == p2:
                side = "op"
            else:
                raise ValueError("SS DEF - unexpected player/card ownership")
            
            # Programatic keys
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"
            gy_key = f"{side}_GY"
            banished_key = f"{side}_banished"
            add_owner = False
            print(start_location,finish_location,side,p1)
            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location in persistent_state[field_key]:
                # field is a dict of zones, flatten values for search
                source = [c for c in persistent_state[field_key].values() if c is not None]
            elif start_location == "Extra Deck" or start_location == "Deck":
                card_name = play["card"]["name"]
                obj_id = play["card"]["object_id"]
            elif start_location == "GY":
                #must check from GY for eff like Goyo
                gy_key = f"{'me' if is_my_card else 'op'}_GY"
                add_owner = True
                source = persistent_state[gy_key]
            elif start_location == "banished":
                source = persistent_state[banished_key]
            else:
                raise ValueError("SS DEF - unhandled start_location")
            if start_location != "Extra Deck" and start_location != "Deck":
                card_entry = next(
                    (c for c in source if c["name"] == card and c["object_id"] == obj_id),
                    None
                )
            else:
                #decompose log for postion and stops hardcode of monster zone
                card_entry = assign_id(p1_me, card_name, "public", obj_id, battle_position=None, face_up=None)
                
            print(source)
            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"
            
            card_entry["state"] = "public"
            card_entry["face_up"] = True
            card_entry["battle_position"] = "DEF"
            if add_owner:
                card_entry["owner"] = f"{'me' if is_my_card else 'op'}"

            if finish_location:
                #once decompose is better split will not be needed
                persistent_state[field_key][finish_location.split(' ')[0]] = card_entry


            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
            if start_location == "GY":
                persistent_state[gy_key] = [
                    c for c in persistent_state[gy_key] if c["object_id"] != obj_id
                ]
            if start_location == "banished":
                persistent_state[banished_key] = [
                    c for c in persistent_state[banished_key] if c["object_id"] != obj_id
                ]
                
        if play["play"] == "To GY":
            card = play["card"]["name"]
            obj_id = play["card"]["object_id"]
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            assert card == card_name, f"Card name mismatch in play {i}: {card} vs {card_name}"
            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = my_card(card, obj_id, all_me_cards)


            if play["username"] == p1 and is_my_card:
                side = "me"
            elif play["username"] == p2 and not is_my_card:
                side = "op"
            else:
                if start_location != "Deck" and start_location != "Extra Deck":
                    raise ValueError("To GY - unexpected player/card ownership")
            
            # Programatic keys
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"
            banish_key = f"{side}_banished"

            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location in persistent_state[field_key]:
                # field is a dict of zones, flatten values for search
                source = [c for c in persistent_state[field_key].values() if c is not None]
            elif start_location == "banished":
                source = persistent_state[banish_key]
            elif start_location == "Deck":
                source = [assign_id(p1_me if side == "me" else p2_op, card, "public", obj_id, battle_position=None, face_up=None)]
            elif start_location == "Extra Deck":
                source = [assign_id(p1_me if side == "me" else p2_op, card, "public", obj_id, battle_position=None, face_up=None)]
            elif start_location == "Field Spell Zone":
                source = [persistent_state[field_key]["FSZ"]]
            else:
                raise ValueError("To GY - unhandled start_location")
            
            card_entry = next(
                (c for c in source if c["name"] == card and c["object_id"] == obj_id),
                None
            )

            # WILL FAIL, BANISH NOT IMPLEMENTED
            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"

            # remove from source
            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
            elif start_location in persistent_state[field_key]:
                for zone, c in persistent_state[field_key].items():
                    if c is not None and c["object_id"] == obj_id:
                        persistent_state[field_key][zone] = None
                        break
            elif start_location == "banished":
                persistent_state[banish_key] = [
                    c for c in persistent_state[banish_key] if c["object_id"] != obj_id
                ]
            elif start_location == "Deck":
                pass  # already handled by creating a new entry
            elif start_location == "Extra Deck":
                pass  # already handled by creating a new entry
            elif start_location == "Field Spell Zone":
                persistent_state[field_key]["FSZ"] = None


            # add to GY
            card_entry["state"] = "public"
            card_entry["face_up"] = None
            #must add to account for different owner
            owner_side = card_entry.get("owner", side)
            persistent_state[f"{'me' if owner_side == 'me' else 'op'}_GY"].append(card_entry)
            card_entry.pop("owner", None)

        if play["play"] == "Mill":
            if play["username"] == p1:
                card_name = play["card"]["name"]
                obj_id = play["card"]["object_id"]
                persistent_state["me_GY"].append(assign_id(p1_me, card_name, "public", obj_id, battle_position=None, face_up=None))
            else:
                card_name = play["card"]["name"]
                obj_id = play["card"]["object_id"]
                persistent_state["op_GY"].append(assign_id(p2_op, card_name, "public", obj_id, battle_position=None, face_up=None))
        
        if play["play"] == "Banish":
            card = play["card"]["name"]
            obj_id = play["card"]["object_id"]
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            assert card == card_name, f"Card name mismatch in play {i}: {card} vs {card_name}"
            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = my_card(card, obj_id, all_me_cards)

            if is_my_card is True:
                side = "me"
            elif is_my_card is False:
                side = "op"
            else:
                if start_location != "Deck":                
                    raise ValueError("Banish - unexpected player/card ownership")
            
            # Programatic keys
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"
            banish_key = f"{side}_banished"
            gy_key = f"{side}_GY"

            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location in persistent_state[field_key]:
                # field is a dict of zones, flatten values for search
                source = [c for c in persistent_state[field_key].values() if c is not None]
            elif start_location == "banished":
                source = persistent_state[banish_key]
            elif start_location == "Deck":
                source = [assign_id(p1_me if side == "me" else p2_op, card, "public", obj_id, battle_position=None, face_up=None)]
            elif start_location == "GY":
                source = persistent_state[gy_key]
            else:
                raise ValueError("To GY - unhandled start_location")
            
            card_entry = next(
                (c for c in source if c["name"] == card and c["object_id"] == obj_id),
                None
            )

            # WILL FAIL, BANISH NOT IMPLEMENTED
            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"

            # remove from source
            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
            elif start_location in persistent_state[field_key]:
                for zone, c in persistent_state[field_key].items():
                    if c is not None and c["object_id"] == obj_id:
                        persistent_state[field_key][zone] = None
                        break
            elif start_location == "banished":
                persistent_state[banish_key] = [
                    c for c in persistent_state[banish_key] if c["object_id"] != obj_id
                ]
            elif start_location == "GY":
                persistent_state[gy_key] = [
                    c for c in persistent_state[gy_key] if c["object_id"] != obj_id
                ]
            elif start_location == "Deck":
                pass  # already handled by creating a new entry


            # add to GY
            card_entry["state"] = "public"
            card_entry["face_up"] = None
            persistent_state[f"{side}_banished"].append(card_entry)
        
        if play["play"] == "To ATK":
            log = get_log(play)
            action, card_name, location, battle_postion = decompose_public_log(log)
            if play["username"] == p1:
                persistent_state["me_field"][location]["battle_position"] = "ATK"
            else:    
                persistent_state["op_field"][location]["battle_position"] = "ATK"

        if play["play"] == "To DEF":
            log = get_log(play)
            action, card_name, location, battle_postion = decompose_public_log(log)
            if play["username"] == p1:
                persistent_state["me_field"][location]["battle_position"] = "DEF"
            else:    
                persistent_state["op_field"][location]["battle_position"] = "DEF"
        
        if play["play"] == "Flip":
            card = play["card"]["name"]
            obj_id = play["card"]["object_id"]
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            assert card == card_name, f"Card name mismatch in play {i}: {card} vs {card_name}"
            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = my_card(card, obj_id, all_me_cards)

            if play["username"] == p1 and is_my_card:
                side = "me"
            elif play["username"] == p2 and not is_my_card:
                side = "op"
            else:
                raise ValueError("ACTIVATE ST - unexpected player/card ownership")
            
            # Programatic keys
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"

            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location in persistent_state[field_key]:
                # field is a dict of zones, flatten values for search
                source = [c for c in persistent_state[field_key].values() if c is not None]
            else:
                raise ValueError("ACTIVATE ST - unhandled start_location")
            
    
            card_entry = next(
                (c for c in source if c["name"] == card and c["object_id"] == obj_id),
                None
            )

            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"

            # update and move card
            card_entry["state"] = "public"
            card_entry["face_up"] = True
            if finish_location:
                persistent_state[field_key][finish_location] = card_entry

            # remove from hand if necessary
            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
        
        if play["play"] == "Flip Summon":
            card = play["card"]["name"]
            obj_id = play["card"]["object_id"]
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            assert card == card_name, f"Card name mismatch in play {i}: {card} vs {card_name}"

            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = my_card(card, obj_id, all_me_cards)

            if play["username"] == p1 and is_my_card:
                side = "me"
            elif play["username"] == p2 and not is_my_card:
                side = "op"
            else:
                raise ValueError("ACTIVATE ST - unexpected player/card ownership")
            
            # Programatic keys
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"

            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location in persistent_state[field_key]:
                # field is a dict of zones, flatten values for search
                source = [c for c in persistent_state[field_key].values() if c is not None]
            else:
                raise ValueError("ACTIVATE ST - unhandled start_location")
            
    
            card_entry = next(
                (c for c in source if c["name"] == card and c["object_id"] == obj_id),
                None
            )

            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"
            
            card_entry["state"] = "public"
            card_entry["battle_position"] = "ATK"
            card_entry["face_up"] = True

            if finish_location:
                persistent_state[field_key][finish_location] = card_entry

            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
        
        if play["play"] == "Summon Token":
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)

            if play["username"] == p1:
                side = "me"
            elif play["username"] == p2:
                side = "op"
            else:
                raise ValueError("Summon Token - unexpected player/card ownership")

            # Programatic keys
            field_key = f"{side}_field"
            
            #all cards need an object id token doesnt get one
            card_entry = assign_id(p1_me,"Token","public",-1,"DEF",True)

            if finish_location:
                persistent_state[field_key][finish_location] = card_entry

        if play["play"] == "Remove Token":
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)

            if play["username"] == p1:
                side = "me"
            elif play["username"] == p2:
                side = "op"
            else:
                raise ValueError("Summon Token - unexpected player/card ownership")

            # Programatic keys
            field_key = f"{side}_field"

            if start_location:
                persistent_state[field_key][start_location] = None
                
        if play["play"] == "Show hand":
            if play["username"] == p1:
                side = "me"
            elif play["username"] == p2:
                side = "op"
            else:
                raise ValueError("Summon Token - unexpected player/card ownership")
            
            hand_key = f"{side}_hand"
            [card.update({'state': 'public'}) for card in persistent_state[hand_key]]
 
        #To T Deck must be implemented(creates problem switching obj ids)
        
        # Compute and display diff for this play    
        after_state = deepcopy(persistent_state)
        diff = json_diff(before_state, after_state)

        if diff:
            print(f"\n🔹 Changes after play {i} ({play['play']}):")
            print(json.dumps(diff, indent=2))
        else:
            print(f"\n(no state change after play {i})")

    # print("\n=== Final Persistent State ===")
    # print(json.dumps(persistent_state, indent=2))
    # with open('final_persistent_state.json', 'w') as f:
    #     json.dump(persistent_state, f, indent=2)


parse(json.load(open(replay, 'r')))